<a href="https://colab.research.google.com/github/ngzh-luke/muict-498prj-ictchat/blob/main/src/model/QA_pair_Augmentor_(Pre_processs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Set device (change to "cpu" if no CUDA GPU available)
device = "cuda"

# Load tokenizer and model from "humarin/chatgpt_paraphraser_on_T5_base"
# Here is the reference to the model
# https://huggingface.co/humarin/chatgpt_paraphraser_on_T5_base
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def paraphrase_question(
  question,
  num_beams=40,  # Increased beam search for exploring more candidates
  num_beam_groups=5,  # Reduced beam groups for focusing on high-scoring beams
  num_return_sequences=5,  # Reduced number of returned paraphrases for efficiency (evaluate more)
  repetition_penalty=10.0,  # Increased penalty to discourage repetitive paraphrases
  diversity_penalty=3.0,  # Lowered penalty to encourage diverse paraphrases
  no_repeat_ngram_size=2,  # Increased n-gram size to penalize longer repeated sequences
  temperature=1.0,  # Increased temperature for more exploration and potentially better results
  do_sample=True,
  max_length=3000
):

  # Prepare the input for the model (optional prompt "paraphrase:" can be removed)
  input_ids = tokenizer(question, return_tensors="pt", padding="longest", max_length=max_length, truncation=True).input_ids.to(device)

  # Generate paraphrases using beam search
  outputs = model.generate(
      input_ids,
      temperature=temperature,
      repetition_penalty=repetition_penalty,
      num_return_sequences=num_return_sequences,
      no_repeat_ngram_size=no_repeat_ngram_size,
      num_beams=num_beams,
      num_beam_groups=num_beam_groups,
      max_length=max_length,
      diversity_penalty=diversity_penalty
  )

  res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  return res

question = "What does MUICT stand for in Mahidol Univerrsity?"
paraphrases = paraphrase_question(question)


In [ ]:
import pandas as pd
import io
from google.colab import files

# Upload file
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='utf-8')


In [ ]:
import re
import time

# Assuming 'df' is already defined and imported

qa_list = []
word_count = 0

start_time = time.time()  # Record the start time

for index, row in df.iterrows():
    question = row['Question']
    answer = row['Answer']

    # Convert multiple lines to single line for the model to read
    question = "".join(question.splitlines())
    answer = "".join(answer.splitlines())

    # Paraphrase the QA pair

    print(f'Q:{question},A:{answer}')
    new_q = paraphrase_question(question)
    new_a = paraphrase_question(answer)

    # Add old question back
    data = {
        'Question': question,
        'Answer': answer
    }
    qa_list.append(data)

    # Split list of QA and append it to the new dataset
    split_q = [item.split(',') for item in new_q if item is not None]
    split_a = [item.split(',') for item in new_a if item is not None]

    for q_item, a_item in zip(split_q, split_a):
        question = q_item
        answer = a_item

        for q_word, a_word in zip(question, answer):
            data = {
                'Question': q_word.strip(),
                'Answer': a_word.strip()
            }
            qa_list.append(data)
            print(data)

            # Count words in question and answer
            word_count += len(q_word.split())
            word_count += len(a_word.split())

print("Total number of words in the document:", word_count)
df = pd.DataFrame(qa_list)

end_time = time.time()  # Record the end time
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")




Q:Can you provide an overview of the Communication Strategies in Professional Life course?,A:Communication Strategies in Professional Life course encompass:Listening, speaking, reading, and writingCommunication in small groups, large groups, and in publicRapport-building with other people in society, including subordinates, supervisors, and colleaguesAppropriate communication methods, including email, telephone, and social networksBuilding effective networksBasic socializationNegotiation and persuasionLeading meetings in a professional mannerLearning negotiation techniquesPresentation tips
{'Question': 'What is the content of Communication Strategies in Professional Life course?', 'Answer': 'Communication Strategies in Professional Life course entail:Listening'}
{'Question': 'Could you explain what the Communication Strategies in Professional Life course entails?', 'Answer': 'Communication Strategies in Professional Life course entail:Listening'}
{'Question': 'What can you tell me abou

In [ ]:
from sklearn.model_selection import train_test_split
# We won't do validiation test set since we use human evaluation

test_size = 0.025
# Split the data into training and test sets
train_set, test_set = train_test_split(df, test_size=test_size, random_state=42)

# Save the training set to a CSV file
train_set.to_csv('training_data.csv', index=False)

# Save the test set to a CSV file
test_set.to_csv('test_data.csv', index=False)

# Download the CSV files
files.download('training_data.csv')
files.download('test_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df